In [11]:
import tensorflow as tf
from os import listdir
import numpy as np
import numpy, os
from PIL import Image
from os import listdir
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [12]:
path="E:/German-Traffic-Signs-Detector/images/train/"

In [13]:
Xlist=[]
Ylist=[]
width = 35
height = 35
for directory in os.listdir(path):
        for file in os.listdir(path+directory):
            img=Image.open(path+directory+"/"+file)
            img = img.resize((width, height)) 
            featurevector = numpy.array(img.getdata())
            #featurevector=featurevector.flatten()
            Xlist.append(featurevector)
            Ylist.append(directory)
Xlist = np.array(Xlist, dtype='float32')
Xlist = np.reshape(Xlist, (Xlist.shape[0], -1))

#Onehot_encode Ylist
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Ylist)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_Ylist = onehot_encoder.fit_transform(integer_encoded)

x_train, x_test, y_train, y_test = train_test_split(Xlist, onehot_Ylist, test_size=0.20, random_state=0)




ntrain=len(x_train)
ntest=len(x_test)
dim= x_train.shape[1]
Totallabel=onehot_Ylist.shape[1]



In [14]:
# Parameters of Logistic Regression
learning_rate   = 0.001
training_epochs = 1000
batch_size      = 10
display_step    = 100

In [15]:
x = tf.placeholder("float", [None, dim]) 
y = tf.placeholder("float", [None, Totallabel]) 
#Set model weights
W = tf.Variable(tf.zeros([dim, Totallabel]))
b = tf.Variable(tf.zeros([Totallabel]))

In [16]:
#Linear mapping
prediction = tf.matmul(x, W) + b
model = tf.nn.softmax(tf.matmul(x, W) + b)

In [18]:
cost_function = -tf.reduce_sum(y*tf.log(prediction)) 
entropy = tf.nn.softmax_cross_entropy_with_logits( logits=prediction, labels=y)
loss = tf.reduce_mean(entropy) # computes the mean over examples in the batch

In [19]:
optm = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss) 

In [20]:
init = tf.global_variables_initializer()

In [21]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_batch = int(ntrain/batch_size)
        # Loop over all batches
        for i in range(num_batch): 
            randidx = np.random.randint(ntrain, size=batch_size)
            batch_xs = x_train[randidx, :]
            batch_ys = y_train[randidx, :]
            # Fit training using batch data
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/num_batch
    predictions = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print ("Accuracy:", accuracy.eval({x: x_test, y: y_test}))
    sess.close()

Accuracy: 0.8106996
